In [2]:
import os
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
from datetime import date

In [3]:
channel_id = 'UCsqjHFMB_JYTaEnf_vmTNqg'
api_key = os.environ['YOUTUBE_API_KEY']

# print(api_key)

# Extract

##### Extract channel stats

In [4]:
def channel_stats(api_key, channel_id):
    channel_stats_url = f"https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2Cstatistics&id={channel_id}&key={api_key}"
    # print(channel_stats_url)
    response = requests.get(channel_stats_url)
    channel_data = json.loads(response.text)
    # print(channel_data)
    return channel_data

##### Extract channel videos

In [5]:
def get_channel_videos(api_key,channel_id):
    page_count = 0
    url = f"https://youtube.googleapis.com/youtube/v3/search?channelId={channel_id}&maxResults=2&order=date&type=video&key={api_key}"


    def get_channel_videos_per_page(url):
        channel_videos = []
    
        response = requests.get(url)
        data = json.loads(response.text)
        nextPageToken = data.get('nextPageToken')
        for i in range(len(data['items'])):
            videoId = data['items'][i]['id']['videoId']
            channel_videos.append(videoId)

        return nextPageToken, channel_videos

    nextPageToken, channel_videos = get_channel_videos_per_page(url)

    while nextPageToken and page_count < 5:
        next_page_url = f"{url}&pageToken={nextPageToken}"
        nextPageToken, new_channel_videos = get_channel_videos_per_page(next_page_url)
        channel_videos.extend(new_channel_videos)
        page_count += 1

    # print(channel_videos)
    return channel_videos

# get_channel_videos(api_key, channel_id)


##### Extract video stats

In [6]:

def get_video_stats(api_key):
    videos_stats = []
    videoIds = get_channel_videos(api_key, channel_id)

    for videoId in videoIds:
        video_data= {}
        video_stats_url = f"https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2Cstatistics&id={videoId}&key={api_key}"
        response = requests.get(video_stats_url)
        data = json.loads(response.text)

        video_data = {
            'publishedAt' : data['items'][0]['snippet']['publishedAt'],
            'title' : data['items'][0]['snippet']['title'],
            'views' : data['items'][0]['statistics']['viewCount'],
            'likes' : data['items'][0]['statistics']['likeCount'],
            'comments' : data['items'][0]['statistics']['commentCount'],

        }
        videos_stats.append(video_data)


    return videos_stats

In [7]:
channel_data = channel_stats(api_key,channel_id)

In [8]:
channel_data

{'kind': 'youtube#channelListResponse',
 'etag': 'nohf55NVWgInmoyacQXr2IQGOh8',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '42jg5aGyKmSRFsDD-IpEECkkfgU',
   'id': 'UCsqjHFMB_JYTaEnf_vmTNqg',
   'snippet': {'title': 'Doug DeMuro',
    'description': '',
    'customUrl': 'dougdemuro',
    'publishedAt': '2013-09-11T19:26:56Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AMLnZu95bYnhPmv4vYZz9QYf-6c6uspJhxZO2cEIDVhHr2M=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AMLnZu95bYnhPmv4vYZz9QYf-6c6uspJhxZO2cEIDVhHr2M=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AMLnZu95bYnhPmv4vYZz9QYf-6c6uspJhxZO2cEIDVhHr2M=s800-c-k-c0x00ffffff-no-rj',
      'width': 800,
      'height': 800}},
    'localized': {'title': 'Doug DeMuro', 'description': ''}},
   'statistics': {'viewCount':

# Transform

##### Tranform channel stats

In [9]:
channel_stats ={
        'channel_id': channel_data['items'][0]['id'],
        'channel_title' : channel_data['items'][0]['snippet']['title'],
        'publishedAt' : channel_data['items'][0]['snippet']['publishedAt'],
        'subscribers' : channel_data['items'][0]['statistics']['subscriberCount'],
        'views' : channel_data['items'][0]['statistics']['viewCount'],
        'videos' : channel_data['items'][0]['statistics']['videoCount']
}

channel_stats_df = pd.DataFrame([channel_stats])

# change datatypes
channel_stats_df['views'] = channel_stats_df['views'].astype(int)
channel_stats_df['videos'] = channel_stats_df['videos'].astype(int)
channel_stats_df['subscribers'] = channel_stats_df['subscribers'].astype(int)
channel_stats_df['publishedAt'] = dt.datetime.strptime(channel_stats_df['publishedAt'].str.split('T')[0][0], '%Y-%m-%d').strftime('%Y-%m-%d')
channel_stats_df

,channel_id,channel_title,publishedAt,subscribers,views,videos
0,UCsqjHFMB_JYTaEnf_vmTNqg,Doug DeMuro,2013-09-11,4450000,1634723071,802


##### Transform video stats

In [10]:
video_data = get_video_stats(api_key)
video_data

[{'publishedAt': '2022-08-04T15:56:18Z',
  'title': 'The 2022 Jeep Grand Cherokee 4xe Is Jeep Going Electric',
  'views': '185563',
  'likes': '6374',
  'comments': '1092'},
 {'publishedAt': '2022-08-02T15:57:01Z',
  'title': 'The McLaren 765LT Spider Is a $500,000 Top-Down Track Monster',
  'views': '396518',
  'likes': '14108',
  'comments': '2016'},
 {'publishedAt': '2022-07-31T15:56:22Z',
  'title': 'The Dodge Ramcharger Is How SUVs Used to Be',
  'views': '522777',
  'likes': '19902',
  'comments': '3624'},
 {'publishedAt': '2022-07-28T15:56:19Z',
  'title': 'The Autech Zagato Stelvio AZ1 Is an Absolutely Bizarre 1980s Creation',
  'views': '565249',
  'likes': '19343',
  'comments': '2658'},
 {'publishedAt': '2022-07-26T15:56:36Z',
  'title': 'The 2022 Land Rover Defender 90 V8 Is Completely Insane',
  'views': '736821',
  'likes': '20835',
  'comments': '2887'},
 {'publishedAt': '2022-07-24T16:12:32Z',
  'title': 'The 2004 Honda Civic Si Is a Quirky, Forgotten Hot Hatchback',
  

In [16]:
video_stats_df = pd.DataFrame(video_data)

#  change datatypes
video_stats_df['comments'] = video_stats_df['comments'].astype(int)
video_stats_df['likes'] = video_stats_df['likes'].astype(int)
video_stats_df['views'] = video_stats_df['views'].astype(int)
video_stats_df['publishedAt'] = pd.to_datetime(video_stats_df['publishedAt'])




# df.dtypes
video_stats_df

,publishedAt,title,views,likes,comments
0,2022-08-04 15:56:18+00:00,The 2022 Jeep Grand Cherokee 4xe Is Jeep Going...,185563,6374,1092
1,2022-08-02 15:57:01+00:00,"The McLaren 765LT Spider Is a $500,000 Top-Dow...",396518,14108,2016
2,2022-07-31 15:56:22+00:00,The Dodge Ramcharger Is How SUVs Used to Be,522777,19902,3624
3,2022-07-28 15:56:19+00:00,The Autech Zagato Stelvio AZ1 Is an Absolutely...,565249,19343,2658
4,2022-07-26 15:56:36+00:00,The 2022 Land Rover Defender 90 V8 Is Complete...,736821,20835,2887
5,2022-07-24 16:12:32+00:00,"The 2004 Honda Civic Si Is a Quirky, Forgotten...",446623,15055,2796
6,2022-07-21 15:54:13+00:00,The Jaguar XJR-15 Is the Unknown $2 Million Su...,584485,22005,3397
7,2022-07-19 15:56:35+00:00,The 2022 Audi R8 Is the End of the Line for th...,692822,21424,3000
8,2022-07-17 15:56:45+00:00,The BMW Alpina B3 Touring Is an Ultra-Cool Obs...,352099,12849,1789
9,2022-07-14 15:56:08+00:00,The Aston Martin Cygnet Is the Most Ridiculous...,1310251,36840,6118


# Load

In [18]:
def dump():
    channel_stats_df.to_csv(f"results/channel_stats/{date.today().strftime('%Y-%m-%d')}.csv", index = False)
    video_stats_df.to_csv(f"results/video_stats/{date.today().strftime('%Y-%m-%d')}.csv", index = False)

    
dump()